In [168]:
import plotly.graph_objects as go
import networkx as nx
import os

# reading input log file

# test_name = "ddmd"
# test_name = "vist"
test_name = "s9f9p8"

stat_path=f"example_stat/{test_name}"
image_path=f"{stat_path}/images"

VFD_ACCESS_SKIP=10

ADD_ADDR=False


In [169]:
# My utility functions
import utils.stat_loader as sload
import utils.stat_print as sp
import utils.vfd_stat2graph as vfd2g
import utils.vfd_graph2sankey as vfd2sk
import utils.full_stat2graph as f2g

# Improve Functions
## TODO
- Add dataset nodes
- Add file address ordering nodes


In [170]:
STAGE_START = 0
STAGE_END = 10


TASK_ORDER_LIST = sload.load_task_order_list(stat_path)
STAGE_END = sload.correct_end_stage(TASK_ORDER_LIST, STAGE_END)

TASK_ORDER_LIST = sload.current_task_order_list(TASK_ORDER_LIST, STAGE_START, STAGE_END)

TASK_LISTS = list(TASK_ORDER_LIST.keys())

print(f"TASK_ORDER_LIST = {TASK_ORDER_LIST}")
TASK_LISTS


STAGE_END is not in TASK_ORDER_LIST, set to max order: 8
TASK_ORDER_LIST = {'run_idfeature': 0, 'run_tracksingle': 1, 'run_gettracks': 2, 'run_trackstats': 3, 'run_identifymcs': 4, 'run_matchpf': 5, 'run_robustmcs': 6, 'run_mapfeature': 7, 'run_speed': 8}


['run_idfeature',
 'run_tracksingle',
 'run_gettracks',
 'run_trackstats',
 'run_identifymcs',
 'run_matchpf',
 'run_robustmcs',
 'run_mapfeature',
 'run_speed']

In [171]:
TIME_STR_START = "Pyflextrkr TIME:"

# Search for log file lines to get workflow time
# Find any file with ".log" extension
log_file = None
for file in os.listdir(stat_path):
    if file.endswith(".log"):
        log_file = os.path.join(stat_path, file)
        break
print(f"log_file = {log_file}")

# Read file and search for workflow time starting with 
with open(log_file, "r") as f:
    for line in f:
        if TIME_STR_START in line:
            WORKFLOW_TIME = float(line.split(" ")[2])
            # Convert from seconds to microseconds
            WORKFLOW_TIME = WORKFLOW_TIME * 1000000
            break

print(f"WORKFLOW_TIME = {WORKFLOW_TIME}")

log_file = example_stat/s9f9p8/test_output.log
WORKFLOW_TIME = 307606284.6183777


In [172]:
vfd_files = sload.find_files_with_pattern(stat_path, "vfd")
# vfd_files = vfd_files[0:1]
print(f"vfd_files: {vfd_files}")

vfd_dict = sload.load_stat_json(vfd_files)
# print(vfd_dict)


vol_files = sload.find_files_with_pattern(stat_path, "vol")
# vol_files = vol_files[0:1]
print(f"vol_files: {vol_files}")

vol_dict = sload.load_stat_json(vol_files)

print("loading json done")

vfd_files: ['example_stat/s9f9p8/71970-vfd_data_stat.json', 'example_stat/s9f9p8/71966-vfd_data_stat.json', 'example_stat/s9f9p8/71959-vfd_data_stat.json', 'example_stat/s9f9p8/71974-vfd_data_stat.json', 'example_stat/s9f9p8/71969-vfd_data_stat.json', 'example_stat/s9f9p8/71964-vfd_data_stat.json', 'example_stat/s9f9p8/71977-vfd_data_stat.json', 'example_stat/s9f9p8/71961-vfd_data_stat.json', 'example_stat/s9f9p8/71911-vfd_data_stat.json']
loading example_stat/s9f9p8/71970-vfd_data_stat.json
loading example_stat/s9f9p8/71966-vfd_data_stat.json
loading example_stat/s9f9p8/71959-vfd_data_stat.json
loading example_stat/s9f9p8/71974-vfd_data_stat.json
loading example_stat/s9f9p8/71969-vfd_data_stat.json
loading example_stat/s9f9p8/71964-vfd_data_stat.json
loading example_stat/s9f9p8/71977-vfd_data_stat.json
loading example_stat/s9f9p8/71961-vfd_data_stat.json
loading example_stat/s9f9p8/71911-vfd_data_stat.json
vol_files: ['example_stat/s9f9p8/71966-vol_data_stat.json', 'example_stat/s9f9p

In [173]:
# Show VFD Tracker overhead
# sp.show_all_overhead("VFD", vfd_dict)
# "VOL": ["VOL-Overhead(us)", "VOL-Init(us)", "VOL-Term(us)", "VOL-Log(us)",
#         "VOL-HT-Overhead(us)", "VOL-HT-Add(us)", "VOL-HT-Rm(us)", "VOL-HT-Search(us)",
#         "VOL-FILE_LL-Overhead(us)", "VOL-FILE_LL-Add(us)", "VOL-FILE_LL-Rm(us)", "VOL-FILE_LL-Update(us)",
#         "VOL-DS_LL-Overhead(us)", "VOL-DS_LL-Add(us)", "VOL-DS_LL-Rm(us)", "VOL-DS_LL-Update(us)",
#         "VOL-GRP_LL-Overhead(us)", "VOL-GRP_LL-Add(us)", "VOL-GRP_LL-Rm(us)",
#         "VOL-DT_LL-Overhead(us)", "VOL-DT_LL-Add(us)", "VOL-DT_LL-Rm(us)",
#         "VOL-All_LL-Overhead(us)"],
# ["VFD-Overhead(us)", ]
        
def show_vfd_io_time(vfd_dict):
    io_time_type = ["POSIX-READ-Time(us)", "POSIX-WRITE-Time(us)", "POSIX-OPEN-Time(us)", "POSIX-CLOSE-Time(us)", "POSIX-DELETE-Time(us)",
                "MMAP-READ-Time(us)", "MMAP-WRITE-Time(us)", "MMAP-OPEN-Time(us)", "MMAP-CLOSE-Time(us)",]


def get_first_last_file_time(entry, first_file_time, last_file_time):
    if "open_time(us)" in entry.keys():
        task_open_time = entry['open_time(us)']
        if first_file_time == -1 or task_open_time < first_file_time:
            first_file_time = task_open_time
    if "close_time(us)" in entry.keys():
        task_close_time = entry['close_time(us)']
        if last_file_time == -1 or task_close_time > last_file_time:
            last_file_time = task_close_time
    return first_file_time, last_file_time

def show_detail_overhead_old(stat_type, file_dict, task_list, workflow_time=0):
    overhead_dict = {
        "VOL": ["VOL-Init(us)", "VOL-Term(us)", "VOL-Log(us)",
                "VOL-HT-Add(us)", "VOL-HT-Rm(us)", "VOL-HT-Search(us)",
                "VOL-FILE_LL-Add(us)", "VOL-FILE_LL-Rm(us)", "VOL-FILE_LL-Update(us)",
                "VOL-DS_LL-Add(us)", "VOL-DS_LL-Rm(us)", "VOL-DS_LL-Update(us)",
                "VOL-GRP_LL-Add(us)", "VOL-GRP_LL-Rm(us)",
                "VOL-DT_LL-Add(us)", "VOL-DT_LL-Rm(us)"],

        "VFD": ["VFD-Init(us)", "VFD-Term(us)", "VFD-Tracker-Init(us)", "VFD-Stat-Add(us)", "VFD-Stat-Update(us)", "VFD-Stat-Rm(us)", "VFD-Stat-Log(us)"]
    }
    
    
    overhead_time_list = []
    first_file_time = -1
    last_file_time = -1
    
    file_overhead_dict = {}
    
    # initialize the dict with task_list and overhead_type
    for pid_file, pid_stat in file_dict.items():
        file_overhead_dict[pid_file] = {}
        for task in task_list:
            file_overhead_dict[pid_file][task] = {}
            for overhead_type in overhead_dict[stat_type]:
                file_overhead_dict[pid_file][task][overhead_type] = []

    for pid_file, pid_stat in file_dict.items():
        prev_task = None
                
        for entry in pid_stat:
            if "Task" in entry.keys():
                task_time = entry["Task"]
                overhead_time_list.append(task_time)
                if stat_type == "VOL":
                    
                    if prev_task is not None:
                        for overhead_type in overhead_dict[stat_type]:
                            file_overhead_dict[pid_file][prev_task][overhead_type].append(task_time[overhead_type])
                
                elif stat_type == "VFD":
                    task_name = task_time['task_name']
                    task_name = task_name.split("-")[0]
                    for overhead_type in overhead_dict[stat_type]:
                        file_overhead_dict[pid_file][task_name][overhead_type].append(task_time[overhead_type])
                    # Get the other entry.keys() to get first and last task time, ther eis only 2 keys 
                    other_keys = [key for key in entry.keys() if key != "Task"]
                    other_key = other_keys[0]
                    first_file_time, last_file_time = get_first_last_file_time(entry[other_key], first_file_time, last_file_time)
            else:
                
                file_id = list(entry.keys())[0]
                first_file_time, last_file_time = get_first_last_file_time(entry[file_id], first_file_time, last_file_time)
                
                if "task_name" in entry[file_id].keys():
                    task_name = entry[file_id]['task_name']
                    # split task_name-pid to get task_name only
                    task_name = task_name.split("-")[0]
                    prev_task = task_name
    
    
    task_overhead_dict = {}
    # initialize the dict with task_list and overhead_type
    for task in task_list:
        task_overhead_dict[task] = {}
        for overhead_type in overhead_dict[stat_type]:
            task_overhead_dict[task][overhead_type] = 0
    
    for pid_file, overhead_time_dict in file_overhead_dict.items():
        print(f"{pid_file}: {overhead_time_dict}")
        for task in task_list:
            for overhead_type in overhead_time_dict[task]:
                pid_overhead = sum(overhead_time_dict[task][overhead_type])
                task_overhead = task_overhead_dict[task][overhead_type]
                
                # pick only 
                if pid_overhead > task_overhead:
                    task_overhead_dict[task][overhead_type] = pid_overhead
                
    # show task_overhead_dict
    for task in task_list:
        print(f"{task}: {task_overhead_dict[task]}")
    
    # Sum all overhead except VOL-Overhead(us)/VFD-Overhead(us)
    for task in task_list:
        task_overhead_dict[task]["Sum-Overhead(us)"] = 0
        for overhead_type in overhead_dict[stat_type]:
            # Pick the largest value of overhead_type
            # print(f'{task}: {overhead_type}: {overhead_time_dict[task][overhead_type]}')
            task_overhead_dict[task]["Sum-Overhead(us)"] += task_overhead_dict[task][overhead_type]
        print(f'{task}: Sum-Overhead(us): {task_overhead_dict[task]["Sum-Overhead(us)"]}')

    if workflow_time == 0:
        workflow_time = last_file_time - first_file_time # get a estimate of workflow time from first file to last file time
    print(f"Total task time (us): {workflow_time}")
    
    # Sum all tasks overhead
    sum_tasks_overhead = 0
    for task in task_list: sum_tasks_overhead += task_overhead_dict[task]["Sum-Overhead(us)"]
    
    # Show overhead percentage on total task time
    print(f"Sum tasks overhead (us): {sum_tasks_overhead}")
    print(f"Overhead percentage on total task time: {sum_tasks_overhead/workflow_time*100}%")
    
    return file_overhead_dict

print("VFD overhead")
vfd_overhead = show_detail_overhead_old("VFD", vfd_dict, TASK_LISTS, workflow_time=WORKFLOW_TIME) # 

print()
print("VOL overhead")
vol_overhead = show_detail_overhead_old("VOL", vol_dict, TASK_LISTS, workflow_time=WORKFLOW_TIME) 

VFD overhead
example_stat/s9f9p8/71970-vfd_data_stat.json: {'run_idfeature': {'VFD-Init(us)': [35.036, 35.036], 'VFD-Term(us)': [0.0, 0.0], 'VFD-Tracker-Init(us)': [1207.774, 1207.774], 'VFD-Stat-Add(us)': [171.215, 171.215], 'VFD-Stat-Update(us)': [45924.867, 45925.904], 'VFD-Stat-Rm(us)': [0.0, 0.0], 'VFD-Stat-Log(us)': [0.0, 4655.101]}, 'run_tracksingle': {'VFD-Init(us)': [73.033, 73.033, 110.016, 110.016, 121.529, 141.476, 161.415], 'VFD-Term(us)': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 'VFD-Tracker-Init(us)': [1207.774, 1207.774, 1207.774, 1207.774, 1207.774, 1207.774, 1207.774], 'VFD-Stat-Add(us)': [393.974, 393.974, 594.438, 594.438, 709.814, 825.675, 950.688], 'VFD-Stat-Update(us)': [53079.882, 53140.357, 60685.354, 60784.856, 63983.157, 66089.841, 68209.461], 'VFD-Stat-Rm(us)': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 'VFD-Stat-Log(us)': [5769.748, 7090.19, 8868.786, 10546.341, 12677.311, 13891.431, 15550.688]}, 'run_gettracks': {'VFD-Init(us)': [], 'VFD-Term(us)': [], 'VFD-Tracker-